## Algoritmo desegunda optimización 



In [17]:
import json
import numpy as np
import logging
import os
import math
import copy
import random
import string
from datetime import datetime, timezone
from typing import Dict, List, Any, Optional, Tuple, Set

# --- Configuration ---
# Use INFO for standard operation, DEBUG for detailed calculation steps
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# --- File Paths ---
# !!! ADJUST THESE PATHS TO YOUR SYSTEM !!!
RELAY_PAIRS_PATH = "//Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs_optimization.json"
OPTIMIZED_SETTINGS_OUTPUT_PATH = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/raw/second_optimized_relay_values.json"
VERIFICATION_OUTPUT_PATH = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/raw/optimized_scenarios_verification.json" # Added path for verification output

# --- Constants ---
K = 0.14  # IEC Constant
N = 0.02  # IEC Exponent
CTI = 0.2 # Coordination Time Interval (seconds)

# --- Scenario Classification Thresholds ---
TMT_THRESHOLD = -0.7  # Optimize scenarios with TMT worse (more negative) than this
TMT_HIGH_PRIORITY = -2.0 # Escenarios extremadamente problemáticos (TMT < this value implies more significant issues)
MIN_COORDINATION_PCT = 95.0  # Optimize scenarios with coordination percentage below this

# --- Optimization Bounds and Parameters ---
MIN_TDS = 0.05
MAX_TDS = 2.5  # Slightly increased upper limit for flexibility
MIN_PICKUP = 0.05
MAX_PICKUP_FACTOR = 0.9 # Max Pickup allowed = MAX_PICKUP_FACTOR * Ishc (Ensures M > 1)
MAX_TIME = 10.0 # Max realistic operation time (seconds)
MIN_OPERATION_TIME = 0.01 # (10 ms) Minimum realistic relay operation time

# --- Optimization Algorithm Settings ---
MAX_ITERATIONS = 600  # Increased iterations for potentially difficult cases
# Convergence Targets (Strict for achieving good coordination)
TARGET_TMT = -0.001  # Target Total Miscoordination Time (close to zero)
MIN_ALLOWED_INDIVIDUAL_MT = -0.01 # Target for the worst single pair's MT (close to zero)
CONVERGENCE_THRESHOLD_TMT = 0.0005  # Tolerance for TMT improvement stagnation
STAGNATION_LIMIT_NORMAL = 40 # Iterations without improvement to stop (Normal Prio)
STAGNATION_LIMIT_HIGH = 60 # Iterations without improvement to stop (High Prio)

# Objective Function Weights (Heavily prioritize fixing miscoordination)
W_TIME = 0.01  # Small weight for total main relay operating time
W_MT = 60.0  # Very high weight for miscoordination penalty

# Adjustment Step Sizes & Factors (Adaptive based on priority and miscoordination severity)
AGGRESSIVE_MT_THRESHOLD = -CTI * 0.5  # Threshold for more aggressive adjustments (-0.1s MT)
# Aggressive Steps (for MT < AGGRESSIVE_MT_THRESHOLD)
AGGRESSIVE_TDS_BACKUP_FACTOR = 1.15 # Multiplicative increase for backup TDS
AGGRESSIVE_TDS_MAIN_FACTOR = 0.85 # Multiplicative decrease for main TDS
AGGRESSIVE_PICKUP_BACKUP_FACTOR = 1.08 # Multiplicative increase for backup Pickup
AGGRESSIVE_PICKUP_MAIN_FACTOR = 0.92 # Multiplicative decrease for main Pickup
# Normal Steps (Additive for TDS, Multiplicative for Pickup)
NORMAL_TDS_BACKUP_ADD = 0.025 # Additive increase for backup TDS
NORMAL_TDS_MAIN_SUB = 0.015 # Additive decrease for main TDS
NORMAL_PICKUP_BACKUP_FACTOR = 1.015 # Gentle multiplicative increase for backup Pickup
NORMAL_PICKUP_MAIN_FACTOR = 0.985 # Gentle multiplicative decrease for main Pickup

# --- Helper Functions ---

def load_json_file(file_path: str) -> Optional[Any]:
    """Loads data from a JSON file with error handling."""
    try:
        with open(file_path, 'r', encoding='utf-8') as file: # Added encoding
            data = json.load(file)
        logger.info(f"Archivo cargado exitosamente: {file_path}")
        return data
    except FileNotFoundError:
        logger.error(f"¡Error Crítico! Archivo no encontrado: {file_path}")
        return None
    except json.JSONDecodeError as e:
        logger.error(f"Error decodificando JSON en {file_path}: {e}")
        return None
    except Exception as e:
        logger.error(f"Error inesperado al cargar {file_path}: {e}", exc_info=True)
        return None

def calculate_operation_time(I_shc: float, I_pi: float, TDS: float) -> Optional[float]:
    """
    Calculates relay operation time using IEC standard inverse time curve.
    Returns None if calculation is impossible or inputs are invalid.
    Ensures a minimum positive operation time (MIN_OPERATION_TIME).
    Returns MAX_TIME for non-operation conditions (I_pi >= I_shc, M <= 1) or invalid calculations.
    """
    # Basic Input Validation
    if not (isinstance(I_shc, (int, float)) and I_shc > 0):
        # logger.debug(f"Invalid I_shc: {I_shc}. Returning None.") # Can be noisy
        return None
    if not (isinstance(I_pi, (int, float)) and I_pi >= MIN_PICKUP):
        # logger.debug(f"Invalid I_pi: {I_pi}. Returning None.")
        return None
    if not (isinstance(TDS, (int, float)) and MIN_TDS <= TDS <= MAX_TDS):
        # logger.debug(f"Invalid TDS: {TDS}. Returning None.")
        return None

    # --- Non-operation conditions ---
    # Pickup Constraint Check: If pickup is >= fault current, relay doesn't pick up.
    if I_pi >= I_shc:
        # logger.debug(f"Non-operation: Pickup {I_pi:.3f} >= I_shc {I_shc:.3f}. Returning MAX_TIME.")
        return MAX_TIME

    # Calculate M = I_shc / I_pi (Current multiple of pickup setting)
    M = I_shc / I_pi

    # Check for M <= 1: Fault current is at or below pickup, relay doesn't operate.
    if M <= 1.0:
        # logger.debug(f"Non-operation: M={M:.4f} <= 1.0. Returning MAX_TIME.")
        return MAX_TIME

    # --- IEC Formula Calculation ---
    try:
        # Calculate the denominator: M^N - 1
        denominator = M**N - 1

        # Check for near-zero denominator (shouldn't happen if M > 1, but for safety)
        if abs(denominator) < 1e-12:
            logger.warning(f"Denominator M**N - 1 is extremely close to zero ({denominator:.2e}) for M={M:.4f}. Returning MAX_TIME.")
            return MAX_TIME

        # Calculate the theoretical time: TDS * [ K / (M^N - 1) ]
        time = TDS * (K / denominator)

        # --- Post-Calculation Checks ---
        # Check for calculation errors (NaN, infinity, negative)
        if not np.isfinite(time) or time <= 0:
            logger.warning(f"Calculated time is non-positive or non-finite ({time}) for I_shc={I_shc:.2f}, I_pi={I_pi:.3f}, TDS={TDS:.3f}. Returning MAX_TIME.")
            return MAX_TIME

        # Enforce minimum operation time
        if time < MIN_OPERATION_TIME:
            # logger.debug(f"Clamping time: Calculated {time:.6f}s < {MIN_OPERATION_TIME}s. Returning {MIN_OPERATION_TIME}s.")
            return MIN_OPERATION_TIME

        # Return the valid calculated time, capped at MAX_TIME
        return min(time, MAX_TIME)

    except OverflowError:
        # If M**N results in overflow, it means M is very large -> time is extremely small.
        logger.warning(f"OverflowError during time calculation (M={M:.4f}, N={N}). Fault current likely very high. Returning MIN_OPERATION_TIME.")
        return MIN_OPERATION_TIME
    except ValueError as e:
        # Should not happen with input checks, but capture just in case
        logger.error(f"ValueError during time calculation (I_shc={I_shc:.2f}, I_pi={I_pi:.3f}, TDS={TDS:.3f}, M={M:.4f}): {e}", exc_info=True)
        return None # Indicate failure
    except Exception as e:
        logger.error(f"Unexpected exception in calculate_operation_time (I_shc={I_shc:.2f}, I_pi={I_pi:.3f}, TDS={TDS:.3f}): {e}", exc_info=True)
        return None # Indicate failure

def calculate_tmt_for_scenario(relay_pairs: List[Dict]) -> Dict:
    """
    Calculates initial TMT and coordination metrics for scenario classification
    using the Time_out values directly from the input JSON data.
    """
    tmt = 0.0
    valid_pairs_for_calc = 0
    coordinated_pairs = 0
    uncoordinated_pairs = 0
    worst_mt = 0.0 # Initialize worst MT to 0

    for pair in relay_pairs:
        main_relay_info = pair.get('main_relay', {})
        backup_relay_info = pair.get('backup_relay', {})

        if not isinstance(main_relay_info, dict) or not isinstance(backup_relay_info, dict):
            continue # Skip malformed pairs

        # Use Time_out from the input file for classification
        main_time = main_relay_info.get('Time_out')
        backup_time = backup_relay_info.get('Time_out')

        # Ensure original times are valid numbers > 0
        if not (isinstance(main_time, (int, float)) and main_time > 0 and
                isinstance(backup_time, (int, float)) and backup_time > 0):
            # logger.debug(f"Skipping pair in TMT calc due to invalid initial Time_out: {main_relay_info.get('relay')}->{backup_relay_info.get('relay')}")
            continue # Skip pairs with invalid initial times

        # If times seem excessively large, they might indicate issues, but we calculate MT anyway for classification
        # if main_time >= MAX_TIME or backup_time >= MAX_TIME:
            # logger.warning(f"Initial Time_out >= MAX_TIME found for pair: {main_relay_info.get('relay')}->{backup_relay_info.get('relay')}")

        valid_pairs_for_calc += 1
        delta_t = backup_time - main_time
        mt = delta_t - CTI # Miscoordination Time

        if mt >= 0:
            coordinated_pairs += 1
        else:
            uncoordinated_pairs += 1
            worst_mt = min(worst_mt, mt)  # Track the most negative MT
            tmt += mt # Accumulate negative MT values for Total Miscoordination Time

    coordination_pct = (100 * coordinated_pairs / valid_pairs_for_calc) if valid_pairs_for_calc > 0 else 0

    return {
        "tmt": tmt,
        "valid_pairs": valid_pairs_for_calc,
        "coordinated_pairs": coordinated_pairs,
        "uncoordinated_pairs": uncoordinated_pairs,
        "worst_mt": worst_mt,
        "coordination_pct": coordination_pct
    }

def classify_scenarios(relay_pairs_data: List[Dict]) -> Tuple[Dict[str, Dict], Dict[str, Dict]]:
    """
    Classifies scenarios into 'problematic' (require optimization) and 'good'
    (keep original values) based on initial TMT and coordination percentage.
    """
    logger.info(f"Clasificando escenarios (Problemático si TMT < {TMT_THRESHOLD} o Coord < {MIN_COORDINATION_PCT}%)...")

    # 1. Group data by scenario_id
    all_scenarios: Dict[str, List[Dict]] = {}
    for pair in relay_pairs_data:
        scenario_id = pair.get("scenario_id")
        if not scenario_id:
            logger.warning(f"Par omitido por falta de 'scenario_id': {pair.get('main_relay',{}).get('relay')} -> {pair.get('backup_relay',{}).get('relay')}")
            continue

        # Basic validation of pair structure before adding
        main_relay_info = pair.get('main_relay', {})
        backup_relay_info = pair.get('backup_relay', {})
        if not isinstance(main_relay_info, dict) or not isinstance(backup_relay_info, dict) or \
           not main_relay_info.get('relay') or not backup_relay_info.get('relay'):
            logger.warning(f"Par omitido en {scenario_id} por formato inválido o falta de nombre de relé.")
            continue

        if scenario_id not in all_scenarios:
            all_scenarios[scenario_id] = []
        all_scenarios[scenario_id].append(pair)

    # 2. Classify each scenario
    problematic_scenarios: Dict[str, Dict] = {}
    good_scenarios: Dict[str, Dict] = {}

    for scenario_id, pairs_in_scenario in all_scenarios.items():
        if not pairs_in_scenario:
            logger.warning(f"Escenario {scenario_id} no tiene pares válidos para análisis. Omitiendo.")
            continue

        # Calculate initial metrics using original Time_out values
        initial_metrics = calculate_tmt_for_scenario(pairs_in_scenario)
        initial_tmt = initial_metrics["tmt"]
        initial_coord_pct = initial_metrics["coordination_pct"]

        # Determine if scenario is problematic
        is_problematic = (initial_tmt < TMT_THRESHOLD) or (initial_coord_pct < MIN_COORDINATION_PCT)

        # Assign priority based on how bad the TMT is
        priority = "ALTA" if initial_tmt < TMT_HIGH_PRIORITY else "NORMAL"

        # Add scenario to the corresponding dictionary
        if is_problematic:
            logger.info(f"⚠️ Escenario {scenario_id}: TMT={initial_tmt:.4f}, Coord={initial_coord_pct:.1f}% → REQUIERE OPTIMIZACIÓN (Prioridad: {priority})")
            problematic_scenarios[scenario_id] = {
                "pairs": pairs_in_scenario,
                "priority": priority,
                "initial_metrics": initial_metrics # Store initial metrics for reference
            }
        else:
            logger.info(f"✓ Escenario {scenario_id}: TMT={initial_tmt:.4f}, Coord={initial_coord_pct:.1f}% → OK, MANTENER VALORES ORIGINALES.")
            good_scenarios[scenario_id] = {
                "pairs": pairs_in_scenario,
                "initial_metrics": initial_metrics
            }

    # 3. Log summary statistics
    logger.info(f"Clasificación completa: {len(problematic_scenarios)} escenarios requieren optimización, "
               f"{len(good_scenarios)} escenarios mantienen valores originales.")

    high_priority_scenarios = [sid for sid, data in problematic_scenarios.items() if data["priority"] == "ALTA"]
    if high_priority_scenarios:
        logger.info(f"Escenarios de ALTA prioridad (TMT < {TMT_HIGH_PRIORITY}): {', '.join(high_priority_scenarios)}")

    return problematic_scenarios, good_scenarios

def extract_relay_config_from_pairs(relay_pairs: List[Dict]) -> Dict[str, Any]:
    """
    Extracts initial relay settings (TDS, pickup) and formats pair information
    needed for the optimization algorithm from the raw pair data.
    """
    initial_settings: Dict[str, Dict[str, float]] = {}
    pairs_info: List[Dict] = []
    relays_in_scenario: Set[str] = set()
    default_pickup = 0.1 # Default pickup if not found/invalid
    default_tds = 0.1 # Default TDS if not found/invalid

    for pair in relay_pairs:
        main_relay_info = pair.get('main_relay', {})
        backup_relay_info = pair.get('backup_relay', {})

        # Ensure basic structure and names exist
        if not isinstance(main_relay_info, dict) or not isinstance(backup_relay_info, dict): continue
        main_relay = main_relay_info.get('relay')
        backup_relay = backup_relay_info.get('relay')
        if not main_relay or not backup_relay: continue

        # Extract and validate fault currents
        I_shc_main = main_relay_info.get('Ishc')
        I_shc_backup = backup_relay_info.get('Ishc')
        if not (isinstance(I_shc_main, (int, float)) and I_shc_main > 0):
            logger.warning(f"Ishc inválido para relé principal '{main_relay}' en par {main_relay}->{backup_relay}. Omitiendo par.")
            continue
        if not (isinstance(I_shc_backup, (int, float)) and I_shc_backup > 0):
             logger.warning(f"Ishc inválido para relé respaldo '{backup_relay}' en par {main_relay}->{backup_relay}. Omitiendo par.")
             continue

        # Add relays to the set for this scenario
        relays_in_scenario.add(main_relay)
        relays_in_scenario.add(backup_relay)

        # Extract initial settings (TDS, pick_up) ONCE per relay
        for relay_name, relay_info in [(main_relay, main_relay_info), (backup_relay, backup_relay_info)]:
            if relay_name not in initial_settings:
                tds_val = relay_info.get('TDS')
                pickup_val = relay_info.get('pick_up') # Note: key is 'pick_up' in input JSON

                # Validate and store initial settings
                valid_tds = isinstance(tds_val, (int, float)) and tds_val >= MIN_TDS
                valid_pickup = isinstance(pickup_val, (int, float)) and pickup_val >= MIN_PICKUP

                if valid_tds and valid_pickup:
                    initial_settings[relay_name] = {
                        'TDS_initial': float(tds_val),
                        'pickup_initial': float(pickup_val)
                    }
                else:
                    # Store placeholder if invalid, will use defaults later in optimization init
                     initial_settings[relay_name] = {
                        'TDS_initial': default_tds if not valid_tds else float(tds_val),
                        'pickup_initial': default_pickup if not valid_pickup else float(pickup_val),
                        'invalid_initial': True # Flag it
                    }
                     logger.warning(f"Valores iniciales inválidos para '{relay_name}' (TDS: {tds_val}, Pickup: {pickup_val}). Se usarán defaults/valores parciales.")


        # Add formatted pair info needed for optimization calculation
        pairs_info.append({
            "main_relay": main_relay,
            "backup_relay": backup_relay,
            "I_shc_main": float(I_shc_main),
            "I_shc_backup": float(I_shc_backup)
        })

    return {
        "initial_settings": initial_settings,
        "pairs_info": pairs_info,
        "relays": relays_in_scenario
    }

def extract_original_settings(scenario_pairs: List[Dict]) -> Dict[str, Dict[str, float]]:
    """
    Extracts the original, valid TDS and pickup values for relays in a 'good' scenario.
    Returns an empty dict if no valid settings can be extracted.
    """
    original_settings: Dict[str, Dict[str, float]] = {}
    relays_processed: Set[str] = set()

    for pair in scenario_pairs:
        main_relay_info = pair.get('main_relay', {})
        backup_relay_info = pair.get('backup_relay', {})

        if not isinstance(main_relay_info, dict) or not isinstance(backup_relay_info, dict):
            continue

        # Process main and backup relays
        for relay_info in [main_relay_info, backup_relay_info]:
            relay_name = relay_info.get('relay')
            if not relay_name or relay_name in relays_processed:
                continue # Skip if no name or already processed

            tds_val = relay_info.get('TDS')
            pickup_val = relay_info.get('pick_up')

            # Validate original settings before storing
            if (isinstance(tds_val, (int, float)) and tds_val >= MIN_TDS and
                    isinstance(pickup_val, (int, float)) and pickup_val >= MIN_PICKUP):
                original_settings[relay_name] = {
                    # Format to consistent decimal places
                    "TDS": float(f"{tds_val:.5f}"),
                    "pickup": float(f"{pickup_val:.5f}")
                }
            else:
                # If original values are invalid for a 'good' scenario, log critical warning
                logger.critical(f"¡VALORES ORIGINALES INVÁLIDOS EN ESCENARIO 'BUENO'! Relay '{relay_name}' (TDS: {tds_val}, Pickup: {pickup_val}). Este relé será EXCLUIDO.")
                # Do not add this relay to the output for this scenario.
                # If all relays in a "good" scenario have invalid original settings, it won't be saved.

            relays_processed.add(relay_name)

    return original_settings


def run_high_priority_optimization(
    scenario_id: str,
    pairs_info: List[Dict],
    initial_settings: Dict[str, Dict[str, float]],
    relays_in_scenario: Set[str],
    priority: str = "NORMAL"
) -> Optional[Dict[str, Dict[str, float]]]:
    """
    Performs the core optimization loop for a single problematic scenario.
    Adjusts TDS and Pickup settings iteratively to minimize miscoordination (TMT).
    Returns the optimized settings or None if optimization fails.
    """
    is_high_priority = priority == "ALTA"
    log_prefix = f"({scenario_id}{'|ALTA' if is_high_priority else ''})" # Prefix for logs
    logger.info(f"{log_prefix} --- Iniciando optimización ---")

    if not pairs_info:
        logger.warning(f"{log_prefix} No hay pares válidos para optimizar. Abortando.")
        return None

    # 1. Initialize Relay Settings
    relay_settings: Dict[str, Dict[str, float]] = {}
    default_tds = 0.1
    default_pickup = 0.1

    for relay in relays_in_scenario:
        if relay in initial_settings:
            # Use provided initial settings if valid, otherwise use defaults
            init_tds = initial_settings[relay].get('TDS_initial', default_tds)
            init_pickup = initial_settings[relay].get('pickup_initial', default_pickup)
            is_invalid = initial_settings[relay].get('invalid_initial', False)

            relay_settings[relay] = {
                "TDS": max(MIN_TDS, min(MAX_TDS, init_tds)),
                "pickup": max(MIN_PICKUP, init_pickup) # Max pickup check is dynamic based on Ishc
            }
            if is_invalid:
                 logger.debug(f"{log_prefix} Relay '{relay}' inicializado con defaults/parciales debido a valores originales inválidos.")
        else:
            # Should not happen if extraction is correct, but handle defensively
            logger.error(f"{log_prefix} Relay '{relay}' no encontrado en initial_settings! Usando defaults.")
            relay_settings[relay] = {"TDS": default_tds, "pickup": default_pickup}

    # 2. Initial Strategy for High Priority (Optional)
    #    (Can sometimes help convergence but also risk overshooting)
    #    Commented out for now, rely on iterative adjustments.
    # if is_high_priority:
    #     main_relays = {pair["main_relay"] for pair in pairs_info}
    #     backup_relays = {pair["backup_relay"] for pair in pairs_info}
    #     for relay in relays_in_scenario:
    #         # ... (logic to adjust initial TDS based on main/backup role) ...
    #         pass

    # 3. Optimization Loop Variables
    best_settings = copy.deepcopy(relay_settings) # Store the best settings found so far
    best_tmt = float('-inf') # Best TMT found (aim to maximize towards 0)
    best_worst_mt = float('-inf') # Best 'worst MT' found (aim to maximize towards 0)
    best_of = float('inf')  # Best objective function value found (aim to minimize)
    no_improvement_streak = 0 # Counter for stagnation detection

    # --- Optimization Loop ---
    for iteration in range(MAX_ITERATIONS):
        # --- Calculate Performance Metrics for Current Settings ---
        current_tmt = 0.0
        current_worst_mt = 0.0
        miscoordination_penalty = 0.0
        total_main_op_time = 0.0
        coordinated_count = 0
        error_calc_count = 0
        max_time_count = 0
        valid_pairs_this_iter = 0
        pair_results_this_iter: List[Dict] = [] # Store detailed results per pair

        for pair in pairs_info:
            main_relay, backup_relay = pair["main_relay"], pair["backup_relay"]
            I_shc_main, I_shc_backup = pair["I_shc_main"], pair["I_shc_backup"]

            # Get current settings (ensure they exist)
            if main_relay not in relay_settings or backup_relay not in relay_settings:
                logger.error(f"{log_prefix} Iter {iteration+1}: Faltan settings para {main_relay} o {backup_relay}. ¡Error interno!")
                error_calc_count += 1
                continue # Skip pair if settings are missing

            tds_main, pickup_main = relay_settings[main_relay]["TDS"], relay_settings[main_relay]["pickup"]
            tds_backup, pickup_backup = relay_settings[backup_relay]["TDS"], relay_settings[backup_relay]["pickup"]

            # Calculate operation times using the helper function
            main_time = calculate_operation_time(I_shc_main, pickup_main, tds_main)
            backup_time = calculate_operation_time(I_shc_backup, pickup_backup, tds_backup)

            # --- Evaluate Pair Coordination ---
            status = "Error"
            mt = None
            delta_t = None

            if main_time is None or backup_time is None:
                error_calc_count += 1
                status = "Error Cálculo"
                miscoordination_penalty += MAX_TIME * 10 # High penalty for calculation errors
            elif main_time >= MAX_TIME or backup_time >= MAX_TIME:
                max_time_count += 1
                status = "MAX_TIME"
                mt = -MAX_TIME # Treat as worst miscoordination
                miscoordination_penalty += abs(mt)**2 # Penalize MAX_TIME operation
            else:
                # Valid times calculated
                valid_pairs_this_iter += 1
                delta_t = backup_time - main_time
                mt = delta_t - CTI
                total_main_op_time += main_time # Accumulate main time

                if mt >= 0:
                    coordinated_count += 1
                    status = "Coordinado"
                    # Optional: Small penalty even if coordinated but delta_t is large? Not implemented.
                else:
                    # Miscoordinated (mt < 0)
                    status = "Descoordinado"
                    current_tmt += mt # Accumulate negative MT
                    current_worst_mt = min(current_worst_mt, mt) # Track worst MT
                    # Apply penalty based on severity
                    power = 2.0 # Quadratic penalty usually sufficient
                    miscoordination_penalty += abs(mt)**power

            # Store results for this pair
            pair_results_this_iter.append({
                "main_relay": main_relay, "backup_relay": backup_relay,
                "I_shc_main": I_shc_main, "I_shc_backup": I_shc_backup,
                "main_time": main_time, "backup_time": backup_time,
                "delta_t": delta_t, "mt": mt, "status": status
            })

        # --- Calculate Objective Function ---
        current_of = W_MT * miscoordination_penalty + W_TIME * total_main_op_time

        # --- Logging Progress ---
        if (iteration + 1) % 50 == 0 or iteration == 0:
            coord_pct = (100 * coordinated_count / valid_pairs_this_iter) if valid_pairs_this_iter > 0 else 0
            logger.info(f"{log_prefix} Iter {iteration+1}/{MAX_ITERATIONS}: "
                        f"OF={current_of:.3f}, TMT={current_tmt:.4f}, WorstMT={current_worst_mt:.4f}, "
                        f"Coord={coord_pct:.1f}% ({coordinated_count}/{valid_pairs_this_iter}), "
                        f"Errs={error_calc_count}, MaxT={max_time_count}")

        # --- Check for Improvement and Update Best Settings ---
        # Prioritize improving TMT and WorstMT. Only accept a better OF if TMT/WorstMT don't degrade significantly.
        tmt_improved = current_tmt > best_tmt - 1e-6
        worst_mt_improved = current_worst_mt > best_worst_mt - 1e-6
        of_improved = current_of < best_of - 1e-6

        # Conditions to save current settings as the best:
        # 1. Significant improvement in TMT or Worst MT, even if OF slightly worse.
        # 2. Improvement in OF without significant degradation in TMT/Worst MT.
        save_as_best = False
        if (tmt_improved and worst_mt_improved) or \
           (current_tmt > best_tmt + 0.05) or \
           (current_worst_mt > best_worst_mt + 0.01): # Condition 1: Significant coord improvement
            save_as_best = True
        elif of_improved and current_tmt >= best_tmt - 0.02 and current_worst_mt >= best_worst_mt - 0.005: # Condition 2: OF improvement without hurting coord too much
            save_as_best = True

        if save_as_best:
            # logger.debug(f"{log_prefix} Iter {iteration+1}: ✓ Nueva mejor config encontrada. OF={current_of:.3f}, TMT={current_tmt:.4f}, WorstMT={current_worst_mt:.4f}")
            best_settings = copy.deepcopy(relay_settings)
            best_tmt = current_tmt
            best_worst_mt = current_worst_mt
            best_of = current_of
            no_improvement_streak = 0 # Reset stagnation counter
        else:
            no_improvement_streak += 1 # Increment stagnation counter

        # --- Check Convergence ---
        if current_tmt >= TARGET_TMT and current_worst_mt >= MIN_ALLOWED_INDIVIDUAL_MT:
            logger.info(f"{log_prefix} ★ Convergencia ALCANZADA en iter {iteration+1}. "
                        f"TMT={current_tmt:.4f} (≥ {TARGET_TMT}), WorstMT={current_worst_mt:.4f} (≥ {MIN_ALLOWED_INDIVIDUAL_MT})")
            break # Exit loop

        # --- Check Stagnation ---
        stagnation_limit = STAGNATION_LIMIT_HIGH if is_high_priority else STAGNATION_LIMIT_NORMAL
        if no_improvement_streak >= stagnation_limit:
            logger.warning(f"{log_prefix} Optimización estancada ({no_improvement_streak} iter sin mejora guardada). Deteniendo.")
            break # Exit loop


        # --- Adjustment Logic ---
        adjustments_made = False
        next_relay_settings = copy.deepcopy(relay_settings) # Work on a copy

        # Filter and sort miscoordinated pairs (where MT is calculated and < 0)
        miscoordinated_pairs = [p for p in pair_results_this_iter if p["mt"] is not None and p["mt"] < -1e-6] # Tolerance
        miscoordinated_pairs.sort(key=lambda p: p["mt"]) # Sort by worst MT first

        if not miscoordinated_pairs:
             # If no miscoordination but convergence not met (perhaps due to MAX_TIME or errors)
            if current_tmt > TARGET_TMT * 2: # If TMT is reasonably close anyway
                logger.info(f"{log_prefix} Iter {iteration+1}: No hay descoordinaciones activas y TMT ({current_tmt:.4f}) es aceptable. Deteniendo ajustes.")
                break
            else:
                 logger.warning(f"{log_prefix} Iter {iteration+1}: TMT ({current_tmt:.4f}) no óptimo, pero no hay pares descoordinados? Revisar pares con error/MAX_TIME.")
                 # Consider adding logic here to slightly adjust settings based on MAX_TIME pairs if needed
                 break # Stop adjustments if no clear target


        # Focus adjustments on the N worst pairs
        focus_pairs_count = min(len(miscoordinated_pairs), 8 if is_high_priority else 5) # Adjust N pairs
        focus_pairs = miscoordinated_pairs[:focus_pairs_count]

        # --- Apply Adjustments Iteratively for Focus Pairs ---
        for pair_idx, pair_res in enumerate(focus_pairs):
            main_r, backup_r = pair_res["main_relay"], pair_res["backup_relay"]
            I_shc_m, I_shc_b = pair_res["I_shc_main"], pair_res["I_shc_backup"]
            mt = pair_res["mt"]

            # Get settings from the *next* settings dictionary (to accumulate changes within iteration)
            tds_m_curr, pickup_m_curr = next_relay_settings[main_r]["TDS"], next_relay_settings[main_r]["pickup"]
            tds_b_curr, pickup_b_curr = next_relay_settings[backup_r]["TDS"], next_relay_settings[backup_r]["pickup"]

            # Determine adjustment factors based on miscoordination severity
            severity_factor = min(1.0, abs(mt) / abs(AGGRESSIVE_MT_THRESHOLD)) # 0 to 1+
            priority_weight = (1.0 - (pair_idx / focus_pairs_count)) # 1.0 down to ~0

            # Proposed new values
            tds_m_new, pickup_m_new = tds_m_curr, pickup_m_curr
            tds_b_new, pickup_b_new = tds_b_curr, pickup_b_curr

            if mt < AGGRESSIVE_MT_THRESHOLD:
                # Aggressive Adjustment (Multiplicative)
                factor = AGGRESSIVE_TDS_BACKUP_FACTOR if is_high_priority else (1 + (AGGRESSIVE_TDS_BACKUP_FACTOR-1)*severity_factor)
                tds_b_new = tds_b_curr * (1 + (factor-1) * priority_weight)

                factor = AGGRESSIVE_TDS_MAIN_FACTOR if is_high_priority else (1 - (1-AGGRESSIVE_TDS_MAIN_FACTOR)*severity_factor)
                tds_m_new = tds_m_curr * (1 - (1-factor) * priority_weight)

                # Adjust pickup more aggressively if TDS hits limits
                if tds_b_new > MAX_TDS * 0.9 or tds_m_new < MIN_TDS * 1.1:
                    factor = AGGRESSIVE_PICKUP_BACKUP_FACTOR if is_high_priority else (1 + (AGGRESSIVE_PICKUP_BACKUP_FACTOR-1)*severity_factor)
                    pickup_b_new = pickup_b_curr * (1 + (factor-1) * priority_weight)

                    factor = AGGRESSIVE_PICKUP_MAIN_FACTOR if is_high_priority else (1 - (1-AGGRESSIVE_PICKUP_MAIN_FACTOR)*severity_factor)
                    pickup_m_new = pickup_m_curr * (1 - (1-factor) * priority_weight)
            else:
                 # Normal Adjustment (Additive TDS, Multiplicative Pickup)
                 tds_change_b = NORMAL_TDS_BACKUP_ADD * severity_factor * priority_weight * (1.5 if is_high_priority else 1.0)
                 tds_change_m = NORMAL_TDS_MAIN_SUB * severity_factor * priority_weight * (1.5 if is_high_priority else 1.0)
                 tds_b_new = tds_b_curr + tds_change_b
                 tds_m_new = tds_m_curr - tds_change_m

                 factor = NORMAL_PICKUP_BACKUP_FACTOR if is_high_priority else (1 + (NORMAL_PICKUP_BACKUP_FACTOR-1)*severity_factor)
                 pickup_b_new = pickup_b_curr * (1 + (factor-1) * priority_weight)
                 factor = NORMAL_PICKUP_MAIN_FACTOR if is_high_priority else (1 - (1-NORMAL_PICKUP_MAIN_FACTOR)*severity_factor)
                 pickup_m_new = pickup_m_curr * (1 - (1-factor) * priority_weight)


            # --- Apply proposed changes with bounds ---
            # Backup Relay
            next_relay_settings[backup_r]["TDS"] = min(MAX_TDS, max(MIN_TDS, tds_b_new))
            max_pickup_b = I_shc_b * MAX_PICKUP_FACTOR # Dynamic max pickup
            next_relay_settings[backup_r]["pickup"] = min(max_pickup_b, max(MIN_PICKUP, pickup_b_new))

            # Main Relay
            next_relay_settings[main_r]["TDS"] = min(MAX_TDS, max(MIN_TDS, tds_m_new))
            max_pickup_m = I_shc_m * MAX_PICKUP_FACTOR # Dynamic max pickup
            next_relay_settings[main_r]["pickup"] = min(max_pickup_m, max(MIN_PICKUP, pickup_m_new))

            # Check if actual changes were made after applying bounds
            if abs(next_relay_settings[backup_r]["TDS"] - tds_b_curr) > 1e-6 or \
               abs(next_relay_settings[backup_r]["pickup"] - pickup_b_curr) > 1e-6 or \
               abs(next_relay_settings[main_r]["TDS"] - tds_m_curr) > 1e-6 or \
               abs(next_relay_settings[main_r]["pickup"] - pickup_m_curr) > 1e-6:
                adjustments_made = True

        # Update main settings if adjustments resulted in changes
        if adjustments_made:
            relay_settings = next_relay_settings
        # If no adjustments were made even though miscoordination exists, it might be stuck at bounds
        elif miscoordinated_pairs:
             logger.debug(f"{log_prefix} Iter {iteration+1}: Miscoordinación existe, pero no se aplicaron ajustes netos (posiblemente en límites).")
             # Let stagnation counter handle this


    # --- End of Optimization Loop ---

    # Final logging based on exit condition
    if current_tmt >= TARGET_TMT and current_worst_mt >= MIN_ALLOWED_INDIVIDUAL_MT:
        logger.info(f"{log_prefix} Optimización finalizada CON convergencia.")
        final_settings_to_use = relay_settings # Use the last settings which converged
        final_tmt_reported = current_tmt
        final_worst_mt_reported = current_worst_mt
    else:
        logger.warning(f"{log_prefix} Optimización finalizada SIN convergencia completa (Iter: {iteration+1}). "
                       f"Mejor TMT: {best_tmt:.4f}, Mejor PeorMT: {best_worst_mt:.4f}")
        if best_tmt <= -MAX_TIME*0.9: # Check if best_tmt was ever updated meaningfully
             logger.error(f"{log_prefix} No se encontró ninguna configuración significativamente mejor que la inicial. Devolviendo None.")
             return None # Return None if optimization essentially failed
        else:
            logger.info(f"{log_prefix} Utilizando la mejor configuración encontrada durante la ejecución.")
            final_settings_to_use = best_settings # Use the best settings found during the run
            final_tmt_reported = best_tmt
            final_worst_mt_reported = best_worst_mt

    # --- Format Final Results ---
    formatted_settings: Dict[str, Dict[str, float]] = {}
    # Pre-calculate max Ishc seen by each relay in this scenario (for final pickup bounding)
    relays_max_ishc: Dict[str, float] = {r: 0.0 for r in relays_in_scenario}
    for p in pairs_info:
        if p["main_relay"] in relays_max_ishc:
             relays_max_ishc[p["main_relay"]] = max(relays_max_ishc[p["main_relay"]], p["I_shc_main"])
        if p["backup_relay"] in relays_max_ishc:
             relays_max_ishc[p["backup_relay"]] = max(relays_max_ishc[p["backup_relay"]], p["I_shc_backup"])

    for relay, settings in final_settings_to_use.items():
        final_tds = settings['TDS']
        final_pickup = settings['pickup']
        max_ishc_for_this_relay = relays_max_ishc.get(relay, MIN_PICKUP / MAX_PICKUP_FACTOR * 1.01) # Default if relay somehow missing

        # Apply final bounds and format
        final_tds_bounded = min(MAX_TDS, max(MIN_TDS, final_tds))
        # Ensure final pickup respects the MAX factor relative to the HIGHEST current seen by this relay
        max_allowable_pickup = max_ishc_for_this_relay * MAX_PICKUP_FACTOR
        final_pickup_bounded = min(max_allowable_pickup, max(MIN_PICKUP, final_pickup))

        formatted_settings[relay] = {
            "TDS": float(f"{final_tds_bounded:.5f}"),
            "pickup": float(f"{final_pickup_bounded:.5f}")
        }

    logger.info(f"{log_prefix} --- Optimización finalizada (TMT final: {final_tmt_reported:.4f}, PeorMT: {final_worst_mt_reported:.4f}) ---")
    return formatted_settings if formatted_settings else None


def verify_optimization_results(
    scenario_id: str,
    pairs_info: List[Dict],
    optimized_settings: Dict[str, Dict[str, float]]
    ) -> Dict:
    """
    Verifies the performance of the final optimized settings for a scenario.
    Recalculates all metrics based on the provided 'optimized_settings'.
    """
    log_prefix = f"(Verif {scenario_id})"
    if not optimized_settings:
        logger.error(f"{log_prefix} No se proporcionaron ajustes optimizados para verificar.")
        return {"scenario_id": scenario_id, "error": "No settings provided"}

    # --- Verification Metrics Initialization ---
    coord_pairs = 0
    uncoord_pairs = 0
    error_pairs = 0
    maxtime_pairs = 0
    total_pairs_in_scenario = len(pairs_info)
    valid_pairs_for_coord_metric = 0 # Count pairs where MT could be calculated
    final_tmt = 0.0
    final_worst_mt = 0.0
    final_total_main_time = 0.0
    detailed_pair_results: List[Dict] = []

    # --- Recalculate Performance for Each Pair ---
    for idx, pair in enumerate(pairs_info):
        main_r, backup_r = pair["main_relay"], pair["backup_relay"]
        I_shc_m, I_shc_b = pair["I_shc_main"], pair["I_shc_backup"]

        # Check if settings exist for this pair
        if main_r not in optimized_settings or backup_r not in optimized_settings:
            logger.warning(f"{log_prefix} Par {idx+1} ({main_r}->{backup_r}): Faltan ajustes finales. Omitiendo.")
            error_pairs += 1 # Count as an error
            continue

        tds_m, pickup_m = optimized_settings[main_r]["TDS"], optimized_settings[main_r]["pickup"]
        tds_b, pickup_b = optimized_settings[backup_r]["TDS"], optimized_settings[backup_r]["pickup"]

        # Recalculate times with final settings
        main_t = calculate_operation_time(I_shc_m, pickup_m, tds_m)
        backup_t = calculate_operation_time(I_shc_b, pickup_b, tds_b)

        # --- Evaluate Final Coordination ---
        status = "Error"
        mt = None
        delta_t = None

        if main_t is None or backup_t is None:
            error_pairs += 1
            status = "Error Cálculo"
        elif main_t >= MAX_TIME or backup_t >= MAX_TIME:
            maxtime_pairs += 1
            status = "MAX_TIME"
            # Decide how to treat MAX_TIME in final TMT: Penalize heavily?
            mt = -MAX_TIME # Penalize
        else:
            # Valid times, calculate coordination
            valid_pairs_for_coord_metric += 1
            delta_t = backup_t - main_t
            mt = delta_t - CTI
            final_total_main_time += main_t

            if mt >= 0:
                coord_pairs += 1
                status = "Coordinado"
            else:
                uncoord_pairs += 1
                status = "Descoordinado"
                final_tmt += mt # Accumulate negative MT
                final_worst_mt = min(final_worst_mt, mt) # Track worst

        # Store detailed results
        detailed_pair_results.append({
            "pair_num": idx + 1,
            "main_relay": main_r, "backup_relay": backup_r,
            "I_shc_main": I_shc_m, "I_shc_backup": I_shc_b,
            "TDS_main": tds_m, "Pickup_main": pickup_m,
            "TDS_backup": tds_b, "Pickup_backup": pickup_b,
            "main_time": main_t if main_t is not None else -1.0,
            "backup_time": backup_t if backup_t is not None else -1.0,
            "delta_t": delta_t, "mt": mt, "status": status
        })

    # --- Calculate Final Metrics ---
    final_coord_pct = (100 * coord_pairs / valid_pairs_for_coord_metric) if valid_pairs_for_coord_metric > 0 else 0

    # Identify worst miscoordinated pairs (excluding errors/MAX_TIME)
    worst_miscoord = sorted(
        [p for p in detailed_pair_results if p["status"] == "Descoordinado"],
        key=lambda x: x["mt"]
    )[:5] # Top 5 worst

    results = {
        "scenario_id": scenario_id,
        "total_pairs_in_scenario": total_pairs_in_scenario,
        "valid_pairs_for_coord_metric": valid_pairs_for_coord_metric,
        "coordinated_pairs": coord_pairs,
        "uncoordinated_pairs": uncoord_pairs, # Only count pairs where MT was calculated < 0
        "error_calculation_pairs": error_pairs,
        "max_time_pairs": maxtime_pairs,
        "final_coordination_percentage": final_coord_pct, # Based on valid pairs
        "final_tmt": final_tmt,
        "final_worst_mt": final_worst_mt,
        "final_total_main_time": final_total_main_time,
        "worst_miscoordinated_pairs_details": worst_miscoord
    }

    logger.info(f"{log_prefix} Verificación Final: Coord={final_coord_pct:.1f}% ({coord_pairs}/{valid_pairs_for_coord_metric}), "
                f"TMT={final_tmt:.4f}, WorstMT={final_worst_mt:.4f}, Err={error_pairs}, MaxT={maxtime_pairs}")

    if worst_miscoord:
         logger.debug(f"{log_prefix} Peores 5 pares descoordinados:")
         for p in worst_miscoord:
             logger.debug(f"  - Par {p['pair_num']}: {p['main_relay']}->{p['backup_relay']}, MT={p['mt']:.4f} (MainT={p['main_time']:.4f}, BackupT={p['backup_time']:.4f})")

    return results


# --- Main Execution Function ---
def main():
    """
    Main function to load data, classify scenarios, run optimization selectively,
    verify results, and save the final settings.
    """
    start_time = datetime.now()
    logger.info(f"--- INICIO: Optimización Selectiva de Ajustes de Relés ({start_time.strftime('%Y-%m-%d %H:%M:%S')}) ---")
    logger.info(f"Usando archivo de pares: {RELAY_PAIRS_PATH}")
    logger.info(f"Guardando ajustes en: {OPTIMIZED_SETTINGS_OUTPUT_PATH}")
    logger.info(f"Guardando verificación en: {VERIFICATION_OUTPUT_PATH}")

    # 1. Load Relay Pair Data
    relay_pairs_data = load_json_file(RELAY_PAIRS_PATH)
    if relay_pairs_data is None or not isinstance(relay_pairs_data, list):
        logger.critical("No se pudieron cargar los datos de pares de relés o el formato es incorrecto. Terminando.")
        raise SystemExit("Error cargando datos de entrada.")
    if not relay_pairs_data:
        logger.warning("El archivo de pares de relés está vacío. No hay nada que procesar.")
        return

    # 2. Classify Scenarios
    try:
        problematic_scenarios, good_scenarios = classify_scenarios(relay_pairs_data)
    except Exception as e:
        logger.critical(f"Error fatal durante la clasificación de escenarios: {e}", exc_info=True)
        raise SystemExit("Fallo irrecuperable en clasificación.")

    if not problematic_scenarios and not good_scenarios:
        logger.warning("No se encontraron escenarios válidos después de la clasificación inicial.")
        # Decide if processing should stop or continue (maybe save empty files?)
        # For now, we'll continue to potentially save empty files if needed.

    # 3. Process Scenarios and Store Final Settings
    all_final_settings: Dict[str, Dict[str, Dict[str, float]]] = {} # {scenario_id: {relay_name: {TDS:val, pickup:val}}}
    verification_results_optimized: List[Dict] = [] # Store verification dicts for optimized scenarios
    scenarios_kept_original_count = 0
    successful_optimizations_count = 0
    failed_optimizations_count = 0

    # 3a. Process 'Good' Scenarios (Keep Original Settings)
    logger.info("\n--- Procesando Escenarios 'Buenos' (Mantener Originales) ---")
    for scenario_id, data in good_scenarios.items():
        try:
            original_settings = extract_original_settings(data["pairs"])
            # Check if *any* valid original settings were extracted for this scenario
            if original_settings:
                # Further check: ensure all relays participating in pairs have settings
                relays_in_good_pairs = set()
                for p in data["pairs"]:
                     main_r = p.get('main_relay', {}).get('relay')
                     bak_r = p.get('backup_relay', {}).get('relay')
                     if main_r: relays_in_good_pairs.add(main_r)
                     if bak_r: relays_in_good_pairs.add(bak_r)

                if all(r in original_settings for r in relays_in_good_pairs):
                    all_final_settings[scenario_id] = original_settings
                    scenarios_kept_original_count += 1
                    # logger.info(f"✓ Escenario {scenario_id}: Se conservan valores originales válidos.")
                else:
                    missing = relays_in_good_pairs - original_settings.keys()
                    logger.warning(f"✗ Escenario bueno {scenario_id}: Se conservan valores originales, PERO faltan ajustes para algunos relés ({missing}). Se omitirán del archivo final.")
            else:
                logger.warning(f"✗ Escenario bueno {scenario_id}: No se pudieron extraer valores originales válidos. Será OMITIDO.")
        except Exception as e:
            logger.error(f"Error procesando escenario bueno '{scenario_id}': {e}", exc_info=True)

    # 3b. Process 'Problematic' Scenarios (Run Optimization)
    logger.info("\n--- Procesando Escenarios 'Problemáticos' (Optimización Requerida) ---")
    # Sort to process High Priority first
    sorted_problematic = sorted(
        problematic_scenarios.items(),
        key=lambda item: 0 if item[1]["priority"] == "ALTA" else 1
    )

    for scenario_id, data in sorted_problematic:
        priority = data["priority"]
        pairs_raw = data["pairs"]
        log_prefix = f"({scenario_id}{'|ALTA' if priority == 'ALTA' else ''})"

        try:
            # Extract config needed for optimization
            config = extract_relay_config_from_pairs(pairs_raw)
            if not config["relays"] or not config["pairs_info"]:
                logger.error(f"{log_prefix} Error crítico: No se pudo extraer configuración válida (relés o pares) para optimización. Omitiendo.")
                failed_optimizations_count += 1
                continue

            # Run the optimization function
            optimized_settings = run_high_priority_optimization(
                scenario_id,
                config["pairs_info"],
                config["initial_settings"],
                config["relays"],
                priority
            )

            # Process optimization results
            if optimized_settings:
                # Verify the results of the optimization
                verification_data = verify_optimization_results(
                    scenario_id,
                    config["pairs_info"],
                    optimized_settings
                )
                # Store results if verification didn't fail
                if "error" not in verification_data:
                    all_final_settings[scenario_id] = optimized_settings
                    verification_results_optimized.append(verification_data)
                    successful_optimizations_count += 1
                    logger.info(f"{log_prefix} Optimización exitosa y verificada.")
                else:
                     logger.error(f"{log_prefix} Optimización produjo settings, pero la verificación falló. Omitiendo.")
                     failed_optimizations_count += 1
            else:
                logger.warning(f"{log_prefix} La optimización no produjo resultados válidos. Omitiendo.")
                failed_optimizations_count += 1

        except Exception as e:
            logger.error(f"{log_prefix} Error inesperado durante la optimización: {e}", exc_info=True)
            failed_optimizations_count += 1


    # 4. Format and Save Results
    logger.info("\n--- Guardando Resultados ---")
    output_data_list = []
    if all_final_settings:
        logger.info(f"Total de escenarios a guardar en el archivo de ajustes: {len(all_final_settings)}")
        for scenario_id, settings_dict in all_final_settings.items():
            if not settings_dict: # Double check for empty settings
                 logger.warning(f"Escenario {scenario_id} tiene settings vacíos al momento de guardar. Omitiendo.")
                 continue

            timestamp_now = datetime.now(timezone.utc).isoformat(timespec='microseconds').replace('+00:00', 'Z')
            output_data_list.append({
                "scenario_id": scenario_id,
                "timestamp": timestamp_now,
                "relay_values": settings_dict # Already formatted {relay: {TDS:v, pickup:v}}
            })
    else:
        logger.warning("No hay ajustes finales (ni originales válidos ni optimizados) para guardar.")


    # 4a. Save Final Settings File (even if empty, indicates process ran)
    try:
        output_dir = os.path.dirname(OPTIMIZED_SETTINGS_OUTPUT_PATH)
        if output_dir: os.makedirs(output_dir, exist_ok=True)

        with open(OPTIMIZED_SETTINGS_OUTPUT_PATH, 'w', encoding='utf-8') as f_out:
            json.dump(output_data_list, f_out, indent=2)
        logger.info(f"Archivo de ajustes guardado en: {OPTIMIZED_SETTINGS_OUTPUT_PATH}")
        if not output_data_list:
             logger.info("(El archivo de ajustes está vacío ya que no hubo resultados válidos)")

    except Exception as e:
        logger.critical(f"¡Error Crítico al guardar archivo de ajustes finales!: {e}", exc_info=True)


    # 4b. Save Verification Results File (only for optimized scenarios)
    if verification_results_optimized:
        try:
            verif_dir = os.path.dirname(VERIFICATION_OUTPUT_PATH)
            if verif_dir: os.makedirs(verif_dir, exist_ok=True)

            with open(VERIFICATION_OUTPUT_PATH, 'w', encoding='utf-8') as f_verif:
                 # Allow NaN/Infinity for internal metrics if they arise, though ideally they shouldn't
                json.dump(verification_results_optimized, f_verif, indent=2, allow_nan=True)
            logger.info(f"Archivo de verificación de optimizados guardado en: {VERIFICATION_OUTPUT_PATH}")
        except Exception as e:
            logger.error(f"Error al guardar archivo de verificación: {e}", exc_info=True)
    else:
        logger.info("No hubo resultados de optimización verificados para guardar.")

    # 5. Final Summary
    end_time = datetime.now()
    duration = end_time - start_time
    logger.info("\n--- RESUMEN FINAL DE EJECUCIÓN ---")
    logger.info(f"Tiempo Total: {duration}")
    logger.info(f"Escenarios Mantenidos Originales: {scenarios_kept_original_count}")
    logger.info(f"Escenarios Optimizados Exitosamente: {successful_optimizations_count}")
    logger.info(f"Escenarios con Fallos/Omitidos: {failed_optimizations_count}")
    total_processed = scenarios_kept_original_count + successful_optimizations_count + failed_optimizations_count
    total_from_classification = len(good_scenarios) + len(problematic_scenarios)
    logger.info(f"Total Escenarios Procesados: {total_processed} (de {total_from_classification} clasificados)")
    logger.info(f"Total Escenarios en Archivo de Salida: {len(output_data_list)}")
    logger.info(f"--- FIN ({end_time.strftime('%Y-%m-%d %H:%M:%S')}) ---")

# --- Entry Point ---
if __name__ == "__main__":
    try:
        main()
    except SystemExit as e:
        logger.critical(f"Ejecución terminada por SystemExit: {e}")
    except Exception as e:
        # Catch any other unexpected critical errors in main
        logger.critical(f"¡Error Crítico Inesperado en main!: {e}", exc_info=True)

2025-04-21 10:43:23,959 - INFO - --- INICIO: Optimización Selectiva de Ajustes de Relés (2025-04-21 10:43:23) ---
2025-04-21 10:43:23,960 - INFO - Usando archivo de pares: //Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs_optimization.json
2025-04-21 10:43:23,960 - INFO - Guardando ajustes en: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/raw/second_optimized_relay_values.json
2025-04-21 10:43:23,960 - INFO - Guardando verificación en: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/raw/optimized_scenarios_verification.json
2025-04-21 10:43:24,024 - INFO - Archivo cargado exitosamente: //Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs_optimization.json
2025-04-21 10:43:24,025 - INFO - Clasificando escenarios (Problemático si TMT < -0.7 o Coord < 95.0%)...
2025-04-21 10:43:24,029 - INFO - ⚠️ Escenario scenario_1: TMT=-0.7393, Coord=87.0%